In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import os
import sys

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a 2022WI folder and put all the files under A4 folder, then "2022WI/A4"
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '/content/drive/MyDrive/UMich Courses/SI 650 EECS 549/Project'
GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

os.chdir(GOOGLE_DRIVE_PATH)
# Add to sys so we can import .py files.
sys.path.append(GOOGLE_DRIVE_PATH)

['scrapped_amazon_general_queries_data.csv', 'combined_queries.csv', 'final_results_references - final_results_references.csv', 'data_T5.csv', 't5_data_generation.ipynb']


In [5]:
import pandas as pd
import itertools

In [13]:
df = pd.read_csv('./scrapped_amazon_general_queries_data.csv', usecols=["productTitle", "productDescription", "Brand", "Ingredients", "About this item", "URL"], sep=';', header=0)
df['docno'] = 'd' + (df.reset_index().index).astype(str)
df['text'] = df[["productTitle", "productDescription", "Brand", "Ingredients", "About this item"]].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df2 = df[["docno","text"]]
doc_dict = {df2['docno'][i]:df2['text'][i] for i in range(len(df2))}
print(df2.head())
df2.to_csv("data_T5.csv")

  docno                                               text
0    d0  Maybelline Hyper Easy Liquid Pen No-Skip Eyeli...
1    d1  L'Oreal Paris Makeup Infallible Never Fail Ori...
2    d2  L'Oreal Paris L’Oreal Paris Makeup Infallible ...
3    d3  HotDeal Lakme Insta Deep Intensee Black Eye Li...
4    d4  Lakme Insta Deep Intense Black Eye Liner 9 ml ...


In [40]:
df_queries = pd.read_csv("./combined_queries.csv")
df_queries['qid'] = df_queries['qid'].astype(str)
query_dict = {df_queries['qid'][i]:df_queries['query'][i] for i in range(len(df_queries))}
df_queries.head()

,qid,query
0,1,facewash for oily skin
1,2,concealer stick
2,3,cruelty free foundation
3,4,gel nail kit
4,5,frizzy hair conditioner


In [20]:
df_rel = pd.read_csv("./final_results_references - final_results_references.csv")
df_rel = df_rel[["qid", "docno", "Relevance (1-5)"]]
df_rel = df_rel.rename(columns={"Relevance (1-5)":"rel"})
query_doc_dict = {(str(df_rel['qid'][i]), str(df_rel['docno'][i])):df_rel['rel'][i] for i in range(len(df_rel))}
df_rel.head()

,qid,docno,rel
0,1,d1082,1
1,1,d1093,3
2,1,d1112,1
3,1,d1115,3
4,1,d1200,1


In [9]:
p1 = df_queries['qid'].to_list()
p2 = df2["docno"].to_list()
p_combined = [p1, p2]
pd_permutate = pd.DataFrame(list(itertools.product(*p_combined)), columns=["qid", "docno"])
pd_permutate.head()

,qid,docno
0,1,d0
1,1,d1
2,1,d2
3,1,d3
4,1,d4


In [29]:
pd_permutate["input_seq"][0] = doc_dict[str(pd_permutate["docno"][0])]

<ipython-input-29-86ec54e89112>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_permutate["input_seq"][0] = doc_dict[str(pd_permutate["docno"][0])]


In [30]:
pd_permutate["input_seq"][0]

'Maybelline Hyper Easy Liquid Pen No-Skip Eyeliner Satin Finish Waterproof Formula Eye Liner Makeup Pitch Black 0.018 Fl; Oz Introducing the liquid liner minus the skips, slips, gaps and struggle. Liquid liner goes hyper easy. With its hyper easy brush tip, It delivers the smoothest glide, fitting to the shape of your eye. The revolutionary flex tip brush and custom hexagonal grip gives optimal control and easy, stable application. Available in Pitch Black, it lasts up to 24 hours with in10se color that doesn\'t smudge. Ophthalmologist tested and suitable for sensitive eyes and contact lenses. Maybelline New York  Ingredients G2015752INGREDIENTS AQUA / WATER / EAUSTYRENE/ACRYLATES COPOLYMERPROPYLENE GLYCOLLAURETH-21PENTYLENE GLYCOLPEG-40 HYDROGENATED CASTOR OILPHENOXYETHANOLAMMONIUM ACRYLATES COPOLYMERCAPRYLYL GLYCOLPPG-2-DECETH-30SODIUM DEHYDROACETATESODIUM LAURYL SULFATEDISODIUM LAURETH SULFOSUCCINATE [+/- MAY CONTAIN / PEUT CONTENIRCI 77266 / BLACK 2] D238371/1  [\' Liquid Eyeliner:

In [41]:
import random
pd_permutate["rel"] = 0.0
pd_permutate["input_seq"] = ''
for i in range(len(pd_permutate)):
  qid = str(pd_permutate["qid"][i])
  docno = str(pd_permutate["docno"][i])
  # print(qid, docno)
  # print(doc_dict[docno])
  pd_permutate["input_seq"][i] = 'Q : ' + query_dict[qid] +' D : ' + doc_dict[docno]
  try:
    pd_permutate["rel"][i] = query_doc_dict[(qid, docno)]
  except:
    pd_permutate["rel"][i] = random.uniform(1,2)

pd_permutate.head()

<ipython-input-41-83c186815b0a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_permutate["input_seq"][i] = 'Q : ' + query_dict[qid] +' D : ' + doc_dict[docno]
<ipython-input-41-83c186815b0a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_permutate["rel"][i] = random.uniform(1,2)
<ipython-input-41-83c186815b0a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_permutate["rel"][i] = query_doc_dict[(qid, docno)]


,qid,docno,rel,input_seq
0,1,d0,1.241146,Q : facewash for oily skin D : Maybelline Hype...
1,1,d1,1.889745,Q : facewash for oily skin D : L'Oreal Paris M...
2,1,d2,1.652828,Q : facewash for oily skin D : L'Oreal Paris L...
3,1,d3,1.380079,Q : facewash for oily skin D : HotDeal Lakme I...
4,1,d4,1.859131,Q : facewash for oily skin D : Lakme Insta Dee...


In [42]:
pd_permutate.to_csv("input_sequence_label_t5.csv")